## 1. 最大熵强化学习
- [Soft Actor-Critic:Off-Policy Maximum Entropy Deep Reinforcement Learning with a Stochastic Actor](https://arxiv.org/pdf/1801.01290)
  - 离线策略算法
  - SAC：基于Soft Q-learning改进而来，使用显示的actor函数


- [Soft Q-learning](https://arxiv.org/pdf/1702.08165)

  - 不学习显示的策略函数，而是使用一个Q函数的玻尔兹曼分布
  - learning expressive energy-based policies for continuous states and actions
  - 增强探索能力

对于一个随机变量X,它的概率密度函数为p，它的熵H定义为：
$$
H(X) = \mathbb{E}_{x\sim p}[-\log{p(x)}]
$$
强化学习中可以使用$H(\pi(*|s))$表示策略$\pi$在状态$s$下的探索能力（随机程度）。


#### 标准强化学习
考虑以下infinite-horizon Markov Decision Process$(S, A, p_s, r)$，状态空间$S$和动作空间$A$是连续的:
- 状态转移方程$p_s: S \times S \times A \rightarrow [0, \infty]$
- 奖励函数$r: S\times A\rightarrow [r_{min}, r_{max}]$
- $\rho_{\pi}(s_t)$和$\rho_{\pi}(s_t, a_t)$分别表示state and state-action marginals of the trajectory distribution induced by a policy $\pi(a_t|s_t)$
标准强化学习目标是学习以下最优策略：
$$
\pi_{std}^{*} = \arg\max_{\pi}\sum_{t}\mathbb{E}_{(s_t,a_t)\sim \rho_{\pi}}[r(s_t,a_t)]
$$

#### 最大熵强化学习
$$
\begin{align}
\pi_{maxEnt}^{*} &= \arg\max_{\pi}\sum_{t}\mathbb{E}_{(s_t,a_t)\sim \rho_{\pi}}[r(s_t,a_t)+\alpha H(\pi(*|s_t))]\\
&= \arg\max_{\pi}\sum_{t}\mathbb{E}_{(s_t)\sim \rho_{\pi}}[\sum r(s_t,a_t)+\alpha H(\pi(*|s_t))]\\
\end{align}
$$
通过最大化每个状态$s_t$的熵从而增加每个状态$s_t$下的策略探索能力。

## 2. SAC：soft actro-critic

#### 动作价值函数Q
SAC算法包含两个动作价值函数$Q_1$、$Q_2$和一个策略函数$\pi$：
- 采用两个Q函数缓解Q值估计过高的问题：每次选择$min(Q_1(s,a),Q_2(s,a))$
- 同时采用目标网络$Q_1^{target}$、$Q_2^{target}$

每个Q网络的损失函数如下，而目标网络更新方式跟DDPG算法中目标网络一样：
$$
\begin{align}
L_{\omega} &=\mathbb{E}_{(s_t,a_t,r_t,s_{t+1}\sim R)}[(Q_{\omega}(s_t,a_t)- (r_t+\gamma(\underset{j=1,2}{min}Q_{\omega_j^{target}}(s_{t+1},a_{t+1}))-\alpha\log{\pi(a_{t+s}|s_{t+1})})))^2] \
\end{align}
$$

#### 策略函数$\pi$
SAC的策略目标可以理解为最大化状态价值函数:
$$
\begin{align}
V(s_t) &= \mathbb{E}_{a_t\sim\pi}[\underset{s_t}{\sum}Q(s_t,a_t) + \alpha H(\pi(*|s_t))]\\
&= \color{red}\mathbb{E}_{a_t\sim\pi}[\underset{s_t}{\sum}[Q(s_t,a_t) - \alpha\log(\pi(a_t|s_t))]]
\end{align}
$$

因此策略网络的最小化目标损失函数为
$$
L_{\pi}(\theta)= \mathbb{E}[\alpha\log(\pi_{\theta}(a_t|s_t))-Q_{\omega}(s_t,a_t)]
$$

对于连续动作问题，SAC策略输出高斯分布的均值和方差

